# GSM-VED: Visco-elasto-damage

In [ ]:
%matplotlib widget
from bmcs_matmod.gsm.gsm_models import GSMVED1D
import matplotlib.pylab as plt
import sympy as sp
import numpy as np

In [ ]:
gsm = GSMVED1D()
gsm_ved = gsm.F_engine

In [ ]:
gsm_ved.dot_eps

In [ ]:
sp.simplify(gsm_ved.F_expr)

In [ ]:
gsm_ved.sig_

In [ ]:
gsm_ved.Sig_, gsm_ved.phi_.diff(gsm_ved.Sig.as_explicit()), gsm_ved.Y_, gsm_ved.Phi_

In [ ]:
(gamma_mech, L_, dL_dS_, dL_dS_A_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, Ox, Ix), Sig_n1, f_n1, R_n1, dR_dA_n1_OI = gsm_ved.Sig_f_R_dR_n1


In [ ]:
gamma_mech

In [ ]:
L_

In [ ]:
sp.simplify(dL_dS_)

In [ ]:
Sig_n1

In [ ]:
R_n1

In [ ]:
delta_A.T

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
mparams = {gsm.E:10, gsm.c:1, gsm.r:1, gsm.S:10, gsm.eps_0:1}
n_t = 100
n_I = 1
eps_max = 2
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_ved.n_Eps), dtype=np.float64)
t_t, eps_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_ved.get_response(eps_n_t, t_t, mparams, 10 )
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
omega_t = Eps_t[:, 0, 0]
sig_tIa = gsm_ved.get_sig(eps_ta[..., np.newaxis], Eps_t, mparams)
sig_t = sig_tIa[:, 0, 0]

In [ ]:
%matplotlib widget
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 5))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='royalblue')

ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='royalblue')

ax_eps.plot(t_t, eps_t, ls='dotted')

plt.tight_layout()
plt.show()

In [ ]:

# Define the symbolic variable
t = sp.symbols('t', real=True)
eps = sp.symbols(r'\varepsilon', real=True)

# Define omega as a function of t
omega = sp.Function('omega')

# Define Y(t), leave symbolic or specify if known
Y = sp.Function('Y')(eps)

# Define the constant S
S = sp.symbols('S', positive=True)

# Define the differential equation
ode = sp.Eq(omega(eps).diff(eps), (1 - omega(eps)) * (Y / S))
display(ode)

# Solve the ODE
general_solution = sp.dsolve(ode, omega(eps))

# Display the general solution
print("General Solution:")
display(general_solution)

# Apply the initial condition omega(0) = 0
# Note: C1 is the integration constant in the solution

# Solve for the constant of integration
C1 = sp.symbols('C1')  # Integration constant
omega_general = general_solution.rhs

# Substitute omega(0) = 0
initial_condition = sp.Eq(omega_general.subs(eps, 0), 0)
#constants = {C1: sp.solve(initial_condition, dict=True)}
constants = {C1:-1}
# Substitute the constant back into the general solution
particular_solution = general_solution.subs(constants)

print("\nParticular Solution with Initial Condition omega(0) = 0:")
particular_solution

In [ ]:
dot_eps_c = sp.Symbol(r'\dot{\varepsilon}_c', real=True)
sp.integrate((E * (dot_eps_c * t)**2), (t, 0, t))